### CF
- I_u是一组由用户u评级的项目;
- I_w是一组由用户w评级的项目;
- U_j是一组给项目j进行评分的用户;
#### 基于用户的协同过滤
- 相似度的度量：用户u和用户w之间的Pearson相关系数（PCC）:
$\mathbf{s}_{wu}=\frac{\sum_{k\in\mathcal{I}_w\cap\mathcal{I}_u}(r_{uk}-\bar{r}_u)(r_{wk}-\bar{r}_w)}{\sqrt{\sum_{k\in\mathcal{I}_w\cap\mathcal{I}_u}(r_{uk}-\bar{r}_u)^2}\sqrt{\sum_{k\in\mathcal{I}_w\cap\mathcal{I}_u}(r_{wk}-\bar{r}_w)^2}}$
其中，−1 ≤ s_wu ≤ 1.
- Top-K most nearest neighbors：
  - Obtain the neighbors of user u where s_wu = 0, i.e., N_u
    - In practice, we usually use a large N_u as candidate users (instead of
all the neighbors) due to the high space cost
  - Obtain a set of top-K nearest neighbors of user u from U_j ∩ N_u (when estimating the rating of ˆruj), i.e., N^j_u ⊆ U_j ∩ N_u with |N^j_u| = K.
- 预测公式：$\hat{r}_{uj}=\bar{r}_{u}+\frac{\sum_{w\in\mathcal{N}_{u}^{j}}s_{wu}(r_{wj}-\bar{r}_{w})}{\sum_{w\in\mathcal{N}_{u}^{j}}s_{wu}}$；
有时，我们会使用以下的预测规则，$\hat{r}_{uj}=\bar{r}_{u}+\frac{\sum_{{w\in\mathcal{N}_{u}^{j}}}s_{wu}(r_{wj}-\bar{r}_{w})}{\sum_{{w\in\mathcal{N}_{u}^{j}}}|s_{wu}|}$.
当N^j_u = ∅默认值为¯ru.
#### 基于物品的协同过滤
- 相似度的度量：$s_{kj}=\frac{\sum_{{u\in\mathcal{U}_{k}\cap\mathcal{U}_{j}}}(r_{uk}-\bar{r}_{u})(r_{uj}-\bar{r}_{u})}{\sqrt{\sum_{{u\in\mathcal{U}_{k}\cap\mathcal{U}_{j}}}(r_{uk}-\bar{r}_{u})^{2}}\sqrt{\sum_{{u\in\mathcal{U}_{k}\cap\mathcal{U}_{j}}}(r_{uj}-\bar{r}_{u})^{2}}}$。
- Top-K most nearest neighbors
  -  Obtain the neighbors of item j where s_kj ！= 0, i.e., N_j;
     -  In practice, we usually use a large N_j as candidate items (instead of all the neighbors) due to the high space cost
  -  Obtain the items rated by user u, i.e., I_u
  -  Obtain a set of top-K nearest neighbors of item j from I_u ∩ N_j (when estimating the rating of ˆr_uj), i.e., N_ju ⊆ I_u ∩ N_j with |N^u_j| = K
  -  K is a parameter needs to be tuned, e.g., K ∈ {20, 30, 40, 50, 100}
- 预测公式：
$\hat{r}_{uj}=\frac{\sum_{k\in\mathcal{N}_j^u}s_{kj}r_{uk}}{\sum_{k\in\mathcal{N}_j^u}s_{kj}}$
Notes:
the default value is ¯ru if N^u_j = ∅
N^u_j is dependent on both item j and user u  
#### 混合协同过滤
- 在混合协同过滤中，将UCF的预测评分与ICF的预测评分相结合，得到混合的预测评分：
$\hat{r}_{uj}=\lambda^{UCF}\hat{r}_{uj}^{UCF}+(1-\lambda^{UCF})\hat{r}_{uj}^{ICF}$
where 0 ≤ $λ^{UCF}$ ≤ 1 is a tradeoff parameter.
#### 注意事项
if ˆr_ui > 5, set it as 5;
if ˆr_ui < 1, set it as 1
#### 评价指标
- MAE：
$MAE=\sum_{{(u,i,r_{ui})\in\mathcal{R}^{te}}}|r_{ui}-\hat{r}_{ui}|/|\mathcal{R}^{te}|$
- RMSE：
$RMSE=\sqrt{\sum_{{(u,i,r_{ui})\in\mathcal{R}^{te}}}(r_{ui}-\hat{r}_{ui})^{2}/|\mathcal{R}^{te}|}$
#### 代码实现
在本问题中，K取为50，$λ^{UCF}$取0.5。


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
from tqdm import tqdm
import numpy as np
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error,mean_absolute_error
# 每个用户的评分的平均值
u_r_averages = {}
# 每个用户评分过的电影ID
u_r_items = {}
#每个电影评分过的用户id列表，读取数据
item_rating_lists = {}
all_data = pd.read_csv("E:/ml-100k/ml-100k/u.data",sep='\t',names=['uid','iid','rating','time'])
train_data = pd.read_csv("E:/ml-100k/ml-100k/u1.base",sep='\t',names=['uid','iid','rating','time'])
test_data = pd.read_csv("E:/ml-100k/ml-100k/u1.test",sep='\t',names=['uid','iid','rating','time'])

for i,group in train_data.groupby('uid'):
    u_r_averages[i] = group['rating'].mean()# 算均值
    u_r_items[i] = set(group['iid'])
# 建立倒排表：电影->用户
for i,group in train_data.groupby('iid'):
    item_rating_lists[i] = set(group['uid'])

ratings = {}
for i,group in train_data.groupby('uid'):
    rating_dict = dict(zip(group['iid'], group['rating']))
    ratings[i] = rating_dict

rating_arr = [[0]*1682 for i in range(943)]
for i,group in train_data.groupby('uid'):
    arr = zip(group['iid'], group['rating'])
    for pair in arr:
        rating_arr[i-1][pair[0]-1] = pair[1]
test_iid_set = set(test_data['iid'])
train_iid_set = set(train_data['iid'])

new_item_set = test_iid_set - train_iid_set

# 示例数据
x = np.array(rating_arr[0])
y = np.array(rating_arr[1])
 
# 计算皮尔逊相关系数
coef, p_value = pearsonr(x,y)
 
#print(f"皮尔逊相关系数: {coef}, 显著性水平p-value: {p_value}")

similar_pcc = np.corrcoef(rating_arr)
'''
ratings列表：是评分表， ratings[i][j]表示用户ID=i+1对电影ID=j+1的评分
u_r_items字典：每个用户评分过的电影ID
u_r_averages字典：每个用户的评分的平均值
''' 
def pcc(ratings, u_r_items, u_r_averages, user_num):
    user_set = u_r_averages.keys()
    pcc_arr = np.ones((user_num, user_num))
    
    for uid in tqdm(user_set):
        for pair in user_set:
            if uid == pair:
                continue
            if pcc_arr[pair-1][uid-1] != 1:
                pcc_arr[uid-1][pair-1] = pcc_arr[pair-1][uid-1]
                continue

            common_rating_items = u_r_items.get(uid) & u_r_items.get(pair)
            if not common_rating_items:
                continue  # No common ratings

            Numerator, denominator_one, denominator_two = 0, 0, 0
            for iid in common_rating_items:
                uid_rating = ratings[uid][iid]
                pair_rating = ratings[pair][iid]
                bias_uid = uid_rating - u_r_averages[uid]
                bias_pair = pair_rating - u_r_averages[pair]
                
                Numerator += bias_uid * bias_pair
                denominator_one += bias_uid**2
                denominator_two += bias_pair**2

            denominator_one = math.sqrt(denominator_one)
            denominator_two = math.sqrt(denominator_two)

            if denominator_one == 0 or denominator_two == 0:
                continue
            
            pcc_value = Numerator / (denominator_one * denominator_two)
            pcc_arr[uid-1][pair-1] = pcc_value
            
    return pcc_arr

def ucf_neigh_select(pcc_arr, item_rating_lists, k, uid, iid):
    recList = {}
    item_rated_users = item_rating_lists.get(iid, set())
    pcc_non_zero = {index + 1 for index, value in enumerate(pcc_arr[uid-1]) if value != 0}
    pcc_non_zero.discard(uid)

    common_users = item_rated_users & pcc_non_zero
    if not common_users:
        return {}

    for i in common_users:
        small_id = min(uid, i)
        big_id = max(uid, i)
        recList[i] = pcc_arr[small_id-1][big_id-1]

    topK = dict(sorted(recList.items(), key=lambda x: x[1], reverse=True)[:k])
    return topK

    
def testUCF(path, pcc_arr, item_rating_lists, k):
    predicts = []
    targets = []
    with open(path, "r") as file:
        lines = file.readlines()
        for i in range(len(lines)):
            predict_u_r = 0
            user_bias = 0
            sum_pcc = 0
            uid = int(lines[i].split()[0])
            iid = int(lines[i].split()[1])
            # 冷启动处理，测试集中的物品在训练集没有出现过，则跳过该物品的评分预测
            if iid in new_item_set:
                continue
            target_rating = int(lines[i].split()[2])
            user_rating_average = u_r_averages.get(uid)
            
            top_k_dict = ucf_neigh_select(pcc_arr, item_rating_lists, k, uid, iid)
            if len(top_k_dict) == 0:
                predict_u_r = user_rating_average
            else:
                for userId,pcc in top_k_dict.items():
                    user_bias += ((ratings.get(userId).get(iid) - u_r_averages.get(userId)) * pcc)
                    sum_pcc += abs(pcc)
                user_bias = user_bias / (sum_pcc)
                predict_u_r = user_rating_average + user_bias
            # 后处理过程
            if predict_u_r > 5:
                predict_u_r = 5
            if predict_u_r < 1:
                predict_u_r = 1
            
            predicts.append(predict_u_r)
            targets.append(target_rating)
    return targets,predicts
# similar_arr = pcc(ratings, u_r_items, u_r_averages,943)
similar_arr = np.corrcoef(rating_arr)
a,b = testUCF("E:/ml-100k/ml-100k/u1.test", similar_arr, item_rating_lists, 50)

from sklearn.metrics import mean_squared_error,mean_absolute_error
print('UCF_RMSE={}'.format(math.sqrt(mean_squared_error(a,b))))
print('UCF_MAE={}'.format(mean_absolute_error(a,b)))

'''
ratings列表：是评分表， ratings[i][j]表示用户ID=i+1对电影ID=j+1的评分
item_rating_lists：每个电影评分过的用户ID
u_r_averages字典：每个用户的评分的平均值
''' 
def acs(ratings, item_rating_lists, u_r_averages, item_num):
    iid_set = item_rating_lists.keys()
    acs_arr = [[1]*item_num for _ in range(item_num)]
    for k in tqdm(iid_set):
        for j in iid_set:
            if k == j:
                continue
            # 如果另一对已经做过，则不用去做了    
            if acs_arr[j-1][k-1] != 1:
                acs_arr[k-1][j-1] = acs_arr[j-1][k-1]
                continue
            numerator = 0
            denominator_one = 0
            denominator_two = 0
            common_rating_users = item_rating_lists.get(k) & item_rating_lists.get(j)
            for uid in common_rating_users:
                bias_k = ratings.get(uid).get(k) - u_r_averages.get(uid)
                bias_j = ratings.get(uid).get(j) - u_r_averages.get(uid)
                numerator += round(bias_k * bias_j,4)
                denominator_one += round(bias_k * bias_k,4)
                denominator_two += round(bias_j * bias_j,4)
            denominator_one = math.sqrt(denominator_one)
            denominator_two = math.sqrt(denominator_two)
            if denominator_one == 0 or denominator_two == 0:
                    continue
            acs_value = numerator / round(denominator_one * denominator_two,4)
            # acs 值在[-1，1]区间            
            if abs(acs_value) > 1:
                acs_value = int(acs_value)
            acs_arr[k-1][j-1] = acs_value
    return acs_arr

'''
acs_arr 列表：acs_arr[k][j] 表示物品k与物品j的相关系数，只记录了上半三角，必须从小索引先开始k<j
item_rating_lists字典：item_rating_lists[i]表示对电影ID=i评分过的用户ID集合
'''
def icf_neigh_select(acs_arr,u_r_items, k, uid, iid):
    recList = {}
    # 用户评过分的物品
    user_rated_items = u_r_items.get(uid)
    # 和物品iid的ACS不等于0的物品ID集合，从1开始的
    acs_non_zero = set([index+1 for index, value in enumerate(acs_arr[iid-1]) if value != 0])
    if user_rated_items == None:
        user_rated_items = set()
    common_items = user_rated_items & acs_non_zero
    if len(common_items)==0:
        return dict()
    for i in common_items:
        # 因为相似度矩阵只保留了上三角，所以要从小的下标开始索引
        small_id = min(iid,i)
        big_id = max(iid,i)
        recList[i] = acs_arr[small_id-1][big_id-1]
    topK = dict(sorted(recList.items(), key=lambda x: x[1],reverse = True))
    if len(topK) > k:
        return dict(list(topK.items())[:k])
    else:
        return topK
def testICF(path, acs_arr, u_r_items, k):
    predicts = []
    targets = []
    with open(path, "r") as file:
        lines = file.readlines()
        for i in range(len(lines)):
            predict_u_r = 0
            user_bias = 0
            sum_acs = 0
            uid = int(lines[i].split()[0])
            iid = int(lines[i].split()[1])
#             if iid in setC:
#                 continue
            target_rating = int(lines[i].split()[2])
            user_rating_average = u_r_averages.get(uid)
            
            top_k_dict = icf_neigh_select(acs_arr, u_r_items, k, uid, iid)
            if len(top_k_dict) == 0:
                predict_u_r = user_rating_average
            else:
                for item_id,acs in top_k_dict.items():
                    user_bias += ratings.get(uid).get(item_id) * acs
                    sum_acs += acs
                # uid=130 iid=1273 top_k_dict = {1278: 1.0, 1279: -1.0}
                if sum_acs == 0:
                    print(uid,iid,top_k_dict)
                    continue
                user_bias = user_bias / (sum_acs)
                predict_u_r = user_bias
            # 后处理过程
            if predict_u_r > 5:
                predict_u_r = 5
            if predict_u_r < 1:
                predict_u_r = 1
            
            predicts.append(predict_u_r)
            targets.append(target_rating)
    return targets,predicts
similar_arr = acs(ratings, item_rating_lists, u_r_averages, 1682)
# a,b = testICF("datasets/ml-100k/u1.test", similar_arr, u_r_items, 50)
from sklearn.metrics.pairwise import cosine_similarity

# a=[[1,3,2],[2,2,1]]
transposed_matrix = list(map(list, zip(*rating_arr)))
result = cosine_similarity(transposed_matrix)

similar_arr[0][0]
import numpy as np
vec1 = np.array(transposed_matrix[0])
vec2 = np.array(transposed_matrix[1])

cos_sim = vec1.dot(vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
#print(cos_sim)

a,b = testICF("E:/ml-100k/ml-100k/u1.test", result, u_r_items, 50)

print('ICF_RMSE={}'.format(math.sqrt(mean_squared_error(a,b))))
print('ICF_MAE={}'.format(mean_absolute_error(a,b)))

from sklearn.metrics import mean_squared_error, mean_absolute_error

# 定义Hybrid CF的测试函数
def testHybridCF(path, ucf_arr, icf_arr, item_rating_lists, u_r_items, k, lambda_val=0.5):
    predicts = []
    targets = []
    with open(path, "r") as file:
        lines = file.readlines()
        for i in range(len(lines)):
            uid = int(lines[i].split()[0])
            iid = int(lines[i].split()[1])
            target_rating = int(lines[i].split()[2])
            user_rating_average = u_r_averages.get(uid)
            
            # 处理UCF部分
            top_k_ucf_dict = ucf_neigh_select(ucf_arr, item_rating_lists, k, uid, iid)
            predict_u_r_ucf = user_rating_average
            user_bias_ucf = 0
            sum_pcc = 0
            if len(top_k_ucf_dict) != 0:
                for userId, pcc in top_k_ucf_dict.items():
                    user_bias_ucf += ((ratings.get(userId).get(iid) - u_r_averages.get(userId)) * pcc)
                    sum_pcc += abs(pcc)
                if sum_pcc != 0:
                    user_bias_ucf = user_bias_ucf / sum_pcc
                    predict_u_r_ucf += user_bias_ucf
            
            # 处理ICF部分
            top_k_icf_dict = icf_neigh_select(icf_arr, u_r_items, k, uid, iid)
            predict_u_r_icf = 0
            user_bias_icf = 0
            sum_acs = 0
            if len(top_k_icf_dict) != 0:
                for item_id, acs in top_k_icf_dict.items():
                    user_bias_icf += ratings.get(uid).get(item_id) * acs
                    sum_acs += acs
                if sum_acs != 0:
                    user_bias_icf = user_bias_icf / sum_acs
                    predict_u_r_icf += user_bias_icf

            # 计算混合预测评分
            predict_u_r_hybrid = lambda_val * predict_u_r_ucf + (1 - lambda_val) * predict_u_r_icf

            # 后处理过程，限制评分范围在[1, 5]之间
            if predict_u_r_hybrid > 5:
                predict_u_r_hybrid = 5
            if predict_u_r_hybrid < 1:
                predict_u_r_hybrid = 1

            predicts.append(predict_u_r_hybrid)
            targets.append(target_rating)

    return targets, predicts

# 使用Hybrid CF进行预测，并计算RMSE和MAE
lambda_val = 0.5  # 设置混合模型的权重
a, b = testHybridCF("E:/ml-100k/ml-100k/u1.test", similar_arr, result, item_rating_lists, u_r_items, 50, lambda_val)
print('Hybrid_CF_RMSE={}'.format(math.sqrt(mean_squared_error(a, b))))
print('Hybrid_CF_MAE={}'.format(mean_absolute_error(a, b)))


UCF_MSE=0.9561858486943877
UCF_MAE=0.7492020495442102


100%|██████████| 1650/1650 [01:30<00:00, 18.22it/s] 


ICF_RMSE=1.0129494935581418
ICF_MAE=0.7953087606829314
Hybrid_CF_RMSE=1.002138013931533
Hybrid_CF_MAE=0.7860205899420444


In [5]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr

# 读取数据
all_data = pd.read_csv("E:/ml-100k/ml-100k/u.data", sep='\t', names=['uid', 'iid', 'rating', 'time'])
train_data = pd.read_csv("E:/ml-100k/ml-100k/u1.base", sep='\t', names=['uid', 'iid', 'rating', 'time'])
test_data = pd.read_csv("E:/ml-100k/ml-100k/u1.test", sep='\t', names=['uid', 'iid', 'rating', 'time'])

# 初始化变量
u_r_averages = {}    # 每个用户的平均评分
u_r_items = {}       # 每个用户评分过的物品ID集合
item_rating_lists = {}  # 每个物品被评分的用户ID集合
ratings = {}         # 用户对物品的评分字典

# 处理训练数据
for uid, group in train_data.groupby('uid'):
    u_r_averages[uid] = group['rating'].mean()
    u_r_items[uid] = set(group['iid'])
    ratings[uid] = dict(zip(group['iid'], group['rating']))

# 建立物品到用户的倒排表
for iid, group in train_data.groupby('iid'):
    item_rating_lists[iid] = set(group['uid'])

# 创建用户-物品评分矩阵
user_num = train_data['uid'].nunique()
item_num = train_data['iid'].nunique()
rating_arr = np.zeros((user_num, item_num))
for uid, group in train_data.groupby('uid'):
    for iid, rating in zip(group['iid'], group['rating']):
        rating_arr[uid - 1][iid - 1] = rating

# 获取新物品集合（测试集中有而训练集中没有的物品）
test_iid_set = set(test_data['iid'])
train_iid_set = set(train_data['iid'])
new_item_set = test_iid_set - train_iid_set

# 定义计算PCC相似度的函数
def pcc(ratings, u_r_items, u_r_averages, user_num):
    user_set = u_r_averages.keys()
    pcc_arr = np.ones((user_num, user_num))
    for uid in tqdm(user_set):
        for pair in user_set:
            if uid >= pair:
                continue
            common_items = u_r_items[uid] & u_r_items[pair]
            if not common_items:
                pcc_arr[uid - 1][pair - 1] = 0
                continue
            numerator, denom_uid, denom_pair = 0, 0, 0
            for iid in common_items:
                uid_rating = ratings[uid][iid]
                pair_rating = ratings[pair][iid]
                bias_uid = uid_rating - u_r_averages[uid]
                bias_pair = pair_rating - u_r_averages[pair]
                numerator += bias_uid * bias_pair
                denom_uid += bias_uid ** 2
                denom_pair += bias_pair ** 2
            denominator = math.sqrt(denom_uid) * math.sqrt(denom_pair)
            if denominator == 0:
                pcc_value = 0
            else:
                pcc_value = numerator / denominator
            pcc_arr[uid - 1][pair - 1] = pcc_value
            pcc_arr[pair - 1][uid - 1] = pcc_value  # 对称矩阵
    return pcc_arr

# 定义UCF的邻居选择函数
def ucf_neigh_select(pcc_arr, item_rating_lists, k, uid, iid):
    recList = {}
    item_rated_users = item_rating_lists.get(iid, set())
    if uid in item_rated_users:
        item_rated_users.remove(uid)
    common_users = item_rated_users
    if not common_users:
        return {}
    for neighbor_uid in common_users:
        similarity = pcc_arr[uid - 1][neighbor_uid - 1]
        if similarity > 0:
            recList[neighbor_uid] = similarity
    topK = dict(sorted(recList.items(), key=lambda x: x[1], reverse=True)[:k])
    return topK

# 定义UCF的测试函数
def testUCF(test_data, pcc_arr, item_rating_lists, k):
    user_ids = []
    item_ids = []
    predicts = []
    targets = []
    for idx, row in test_data.iterrows():
        uid = row['uid']
        iid = row['iid']
        target_rating = row['rating']
        # 冷启动处理
        if iid in new_item_set:
            continue
        user_rating_average = u_r_averages.get(uid, 0)
        top_k_dict = ucf_neigh_select(pcc_arr, item_rating_lists, k, uid, iid)
        if not top_k_dict:
            predict_u_r = user_rating_average
        else:
            numerator = 0
            denominator = 0
            for neighbor_uid, sim in top_k_dict.items():
                neighbor_rating = ratings[neighbor_uid].get(iid, 0)
                neighbor_avg = u_r_averages.get(neighbor_uid, 0)
                numerator += sim * (neighbor_rating - neighbor_avg)
                denominator += abs(sim)
            if denominator == 0:
                predict_u_r = user_rating_average
            else:
                predict_u_r = user_rating_average + numerator / denominator
        # 后处理
        predict_u_r = max(min(predict_u_r, 5), 1)
        user_ids.append(uid)
        item_ids.append(iid)
        predicts.append(predict_u_r)
        targets.append(target_rating)
    return user_ids, item_ids, targets, predicts

# 计算用户相似度矩阵
print("计算用户相似度矩阵...")
similar_arr = pcc(ratings, u_r_items, u_r_averages, user_num)

# 测试UCF
print("测试UCF...")
user_ids_ucf, item_ids_ucf, targets_ucf, predicts_ucf = testUCF(test_data, similar_arr, item_rating_lists, k=50)
rmse_ucf = math.sqrt(mean_squared_error(targets_ucf, predicts_ucf))
mae_ucf = mean_absolute_error(targets_ucf, predicts_ucf)
print(f'UCF RMSE={rmse_ucf}')
print(f'UCF MAE={mae_ucf}')

# 定义ACS相似度计算函数
def acs(ratings, item_rating_lists, u_r_averages, item_num):
    iid_set = item_rating_lists.keys()
    acs_arr = np.ones((item_num, item_num))
    for iid in tqdm(iid_set):
        for pair_iid in iid_set:
            if iid >= pair_iid:
                continue
            common_users = item_rating_lists[iid] & item_rating_lists[pair_iid]
            if not common_users:
                acs_arr[iid - 1][pair_iid - 1] = 0
                continue
            numerator, denom_iid, denom_pair = 0, 0, 0
            for uid in common_users:
                rating_iid = ratings[uid][iid]
                rating_pair = ratings[uid][pair_iid]
                avg_uid = u_r_averages[uid]
                bias_iid = rating_iid - avg_uid
                bias_pair = rating_pair - avg_uid
                numerator += bias_iid * bias_pair
                denom_iid += bias_iid ** 2
                denom_pair += bias_pair ** 2
            denominator = math.sqrt(denom_iid) * math.sqrt(denom_pair)
            if denominator == 0:
                acs_value = 0
            else:
                acs_value = numerator / denominator
            acs_arr[iid - 1][pair_iid - 1] = acs_value
            acs_arr[pair_iid - 1][iid - 1] = acs_value  # 对称矩阵
    return acs_arr

# 定义ICF的邻居选择函数
def icf_neigh_select(acs_arr, u_r_items, k, uid, iid):
    recList = {}
    user_rated_items = u_r_items.get(uid, set())
    if not user_rated_items:
        return {}
    for item_id in user_rated_items:
        similarity = acs_arr[iid - 1][item_id - 1]
        if similarity > 0:
            recList[item_id] = similarity
    topK = dict(sorted(recList.items(), key=lambda x: x[1], reverse=True)[:k])
    return topK

# 定义ICF的测试函数
def testICF(test_data, acs_arr, u_r_items, k):
    user_ids = []
    item_ids = []
    predicts = []
    targets = []
    for idx, row in test_data.iterrows():
        uid = row['uid']
        iid = row['iid']
        target_rating = row['rating']
        # 冷启动处理
        if iid in new_item_set:
            continue
        user_rating_average = u_r_averages.get(uid, 0)
        top_k_dict = icf_neigh_select(acs_arr, u_r_items, k, uid, iid)
        if not top_k_dict:
            predict_u_r = user_rating_average
        else:
            numerator = 0
            denominator = 0
            for item_id, sim in top_k_dict.items():
                rating = ratings[uid].get(item_id, 0)
                numerator += sim * rating
                denominator += abs(sim)
            if denominator == 0:
                predict_u_r = user_rating_average
            else:
                predict_u_r = numerator / denominator
        # 后处理
        predict_u_r = max(min(predict_u_r, 5), 1)
        user_ids.append(uid)
        item_ids.append(iid)
        predicts.append(predict_u_r)
        targets.append(target_rating)
    return user_ids, item_ids, targets, predicts

# 计算物品相似度矩阵
print("计算物品相似度矩阵...")
# 转置用户-物品评分矩阵
transposed_matrix = rating_arr.T
# 使用余弦相似度计算物品相似度矩阵
acs_arr = cosine_similarity(transposed_matrix)

# 测试ICF
print("测试ICF...")
user_ids_icf, item_ids_icf, targets_icf, predicts_icf = testICF(test_data, acs_arr, u_r_items, k=50)
rmse_icf = math.sqrt(mean_squared_error(targets_icf, predicts_icf))
mae_icf = mean_absolute_error(targets_icf, predicts_icf)
print(f'ICF RMSE={rmse_icf}')
print(f'ICF MAE={mae_icf}')

# 对齐UCF和ICF的预测结果
common_indices = []
for idx, (uid_ucf, iid_ucf) in enumerate(zip(user_ids_ucf, item_ids_ucf)):
    for jdx, (uid_icf, iid_icf) in enumerate(zip(user_ids_icf, item_ids_icf)):
        if uid_ucf == uid_icf and iid_ucf == iid_icf:
            common_indices.append((idx, jdx))
            break

# 提取共同的预测和真实值
targets_common = [targets_ucf[idx] for idx, _ in common_indices]
predicts_ucf_common = [predicts_ucf[idx] for idx, _ in common_indices]
predicts_icf_common = [predicts_icf[jdx] for _, jdx in common_indices]

# 计算混合协同过滤的预测
lambda_ucf = 0.5  # UCF的权重
hybrid_predicts = []
for ucf_pred, icf_pred in zip(predicts_ucf_common, predicts_icf_common):
    hybrid_pred = lambda_ucf * ucf_pred + (1 - lambda_ucf) * icf_pred
    hybrid_pred = max(min(hybrid_pred, 5), 1)
    hybrid_predicts.append(hybrid_pred)

# 计算混合协同过滤的RMSE和MAE
rmse_hybrid = math.sqrt(mean_squared_error(targets_common, hybrid_predicts))
mae_hybrid = mean_absolute_error(targets_common, hybrid_predicts)
print(f'Hybrid CF RMSE={rmse_hybrid}')
print(f'Hybrid CF MAE={mae_hybrid}')


IndexError: index 1650 is out of bounds for axis 0 with size 1650

In [9]:
import pandas as pd
from surprise import Dataset, Reader, accuracy
from surprise import KNNBasic
import numpy as np

# 文件路径
train_file = 'E:/ml-100k/ml-100k/u1.base'
test_file = 'E:/ml-100k/ml-100k/u1.test'

# 数据加载
reader = Reader(line_format='user item rating timestamp', sep='\t', rating_scale=(1, 5))
data_train = Dataset.load_from_file(train_file, reader=reader)
data_test = Dataset.load_from_file(test_file, reader=reader)

# 数据分割
trainset = data_train.build_full_trainset()
testset = data_test.build_full_trainset().build_testset()

# 用户基础协同过滤
sim_options_user_based = {'name': 'pearson_baseline', 'user_based': True}
algo_user_based = KNNBasic(k=50, sim_options=sim_options_user_based)
algo_user_based.fit(trainset)
predictions_user_based = algo_user_based.test(testset)

# 物品基础协同过滤
sim_options_item_based = {'name': 'cosine', 'user_based': False}
algo_item_based = KNNBasic(k=50, sim_options=sim_options_item_based)
algo_item_based.fit(trainset)
predictions_item_based = algo_item_based.test(testset)

# 混合模型
def hybrid_prediction(predictions_user_based, predictions_item_based, lambda_UCF):
    predictions = []
    for pred_ub, pred_ib in zip(predictions_user_based, predictions_item_based):
        # 获取真实评分
        r_ui = pred_ub.r_ui
        # 计算混合预测
        est = lambda_UCF * pred_ub.est + (1 - lambda_UCF) * pred_ib.est
        # 添加到预测列表
        predictions.append((pred_ub.uid, pred_ub.iid, r_ui, est, None))
    return predictions

lambda_UCF = 0.5
predictions_hybrid = hybrid_prediction(predictions_user_based, predictions_item_based, lambda_UCF)

# 将预测结果转换成Surprise的Predictions格式
hybrid_predictions = [
    (pred_uid, pred_iid, pred_r_ui, pred_est, None)
    for pred_uid, pred_iid, pred_r_ui, pred_est, _ in predictions_hybrid
]

# 计算RMSE和MAE
rmse_user_based = accuracy.rmse(predictions_user_based, verbose=False)
mae_user_based = accuracy.mae(predictions_user_based, verbose=False)
rmse_item_based = accuracy.rmse(predictions_item_based, verbose=False)
mae_item_based = accuracy.mae(predictions_item_based, verbose=False)
rmse_hybrid = accuracy.rmse(hybrid_predictions, verbose=False)
mae_hybrid = accuracy.mae(hybrid_predictions, verbose=False)

print(f'User-based CF RMSE: {rmse_user_based:.4f}, MAE: {mae_user_based:.4f}')
print(f'Item-based CF RMSE: {rmse_item_based:.4f}, MAE: {mae_item_based:.4f}')
print(f'Hybrid CF RMSE: {rmse_hybrid:.4f}, MAE: {mae_hybrid:.4f}')

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
User-based CF RMSE: 1.0098, MAE: 0.8016
Item-based CF RMSE: 1.0474, MAE: 0.8257
Hybrid CF RMSE: 0.9724, MAE: 0.7785


### CF
#### User-based CF
- Users with similar tastes in the past will have similar tastes in the future：User-based CF
- $\circ$ A user will like some similar items to those he liked before：Item-based CF
- $\begin{array}{l}\bullet\mathcal{I}_u\text{ is a set of items rated by user }u\\\\\bullet\mathcal{I}_w\text{ is a set of items rated by user }w\\\\\bullet\mathcal{U}_j\text{ is a set of users who rated item }j\end{array}$
- Pearson correlation coefficient (PCC) between user $u$ and user $w$,

$$s_{wu}=\frac{\sum_{k\in\mathcal{I}_w\cap\mathcal{I}_u}(r_{uk}-\bar{r}_u)(r_{wk}-\bar{r}_w)}{\sqrt{\sum_{k\in\mathcal{I}_w\cap\mathcal{I}_u}(r_{uk}-\bar{r}_u)^2}\sqrt{\sum_{k\in\mathcal{I}_w\cap\mathcal{I}_u}(r_{wk}-\bar{r}_w)^2}}$$
Notes:
$0-1\leq s_{wu}\leq1$
- Top-K most nearest neighbors
  - Step 1. Obtain the neighbors of user $u$ where $\mathfrak{s}_{wu}\neq0,\mathfrak{i.e.,N}_u$
    - In practice, we usually use a large $N_u$ as candidate users (instead of all the neighbors) due to the high space cost

  - Step 2. Obtain the users who rated item $j$,i.e.,$\mathcal{U}_j$
  - Step 3. Obtain a set of top-K nearest neighbors of user $u$ from $U_j\cap\mathcal{N}_u$ (when estimating the rating of $\hat{r}_uj)$, i.e., $\mathcal{N}_u^j\subseteq\mathcal{U}_j\cap\mathcal{N}_u$ with $|\mathcal{N}_u^j|=K$
- Predicted rating of user $u$ on item $j$,

$$\hat{r}_{uj}=\bar{r}_u+\frac{\sum_{w\in\mathcal{N}_u^j}s_{wu}(r_{wj}-\bar{r}_w)}{\sum_{w\in\mathcal{N}_u^j}s_{wu}}$$

Notes:

- sometimes, we will use the following prediction rule,

  $$\hat{r}_{uj}=\bar{r}_u+\frac{\sum_{w\in\mathcal{N}_u^j}s_{wu}(r_{wj}-\bar{r}_w)}{\sum_{w\in\mathcal{N}_u^j}|s_{wu}|}$$

  - the default value is $\bar{r}_u$ if $\mathcal{N}_u^j=\emptyset$

  - $ \mathcal{N}_u^j$ is dependent on both user $u$ and item $j$
- $\begin{array}{l}\bullet\mathcal{U}_k\text{ is a set of users who rated item }k\\\\\bullet\mathcal{U}_j\text{ is a set of users who rated item }j\\\\\bullet\mathcal{I}_u\text{ is a set of items rated by user }u\end{array}$
- Adjusted Cosine similarity between item $k$ and item $j$,

$$s_{kj}=\frac{\sum_{u\in\mathcal{U}_k\cap\mathcal{U}_j}(r_{uk}-\bar{r}_u)(r_{uj}-\bar{r}_u)}{\sqrt{\sum_{u\in\mathcal{U}_k\cap\mathcal{U}_j}(r_{uk}-\bar{r}_u)^2}\sqrt{\sum_{u\in\mathcal{U}_k\cap\mathcal{U}_j}(r_{uj}-\bar{r}_u)^2}}$$

Notes

  - $-1\leq s_{kj}\leq1$

  - Cosine similarity between item $k$ and item $j$

$$s_{kj}=\frac{\sum_{u\in\mathcal{U}_k\cap\mathcal{U}_j}r_{uk}r_{uj}}{\sqrt{\sum_{u\in\mathcal{U}_k\cap\mathcal{U}_j}r_{uk}^2}\sqrt{\sum_{u\in\mathcal{U}_k\cap\mathcal{U}_j}r_{uj}^2}}$$
#### Item-based CF
- Similarity threshold:

Top-K most nearest neighbors

  -  Step 1. Obtain the neighbors of item $j$ where s$_kj\neq0$,i.e., $\mathcal{N}_j$

     - In practice, we usually use a large $\mathcal{N}_j$ as candidate items (instead of

all the neighbors) due to the high space cost

  - Step 2. Obtain the items rated by user $u$, i.e., $I_u$

  - Step 3. Obtain a set of top-$K$ nearest neighbors of item $j$ from
$\mathcal{I}_u\cap\mathcal{N}_j$ (when estimating the rating of $\hat{r}_uj)$, i.e., $\mathcal{N}_j^u\subseteq\mathcal{I}_u\cap\mathcal{N}_j$ with
$|\tilde{\mathcal{N}}_j^u|=\dot{K}$

  - K is a parameter needs to be tuned, e.g., $K\in\{20,30,40,50,100\}$
- Predicted rating of user $u$ on item $j$,

$$\hat{r}_{uj}=\frac{\sum_{k\in\mathcal{N}_j^u}s_{kj}r_{uk}}{\sum_{k\in\mathcal{N}_j^u}s_{kj}}$$

  - $\mathsf{Notes:}$
the default value is $\bar{r}_u$ if $\mathcal{N}_j^u=\emptyset$
  - $\mathcal{N}_j^u$ is dependent on both item $j$ and user $u$
#### Hybrid CF
- $\begin{aligned}&\text{Predicted rating of user }u\text{ on item }j,\\&\hat{r}_{uj}=\lambda^{UCF}\hat{r}_{uj}^{UCF}+(1-\lambda^{UCF})\hat{r}_{uj}^{ICF}\\\\&\mathrm{where~}0\leq\lambda^{UCF}\leq1\text{ is a tradeoff parameter.}\end{aligned}$
#### 结果评估
Mean Absolute Error (MAE)

$$MAE=\sum_{(u,i,r_{ui})\in\mathcal{R}^{te}}|r_{ui}-\hat{r}_{ui}|/|\mathcal{R}^{te}|$$


 Root Mean Square Error (RMSE)

$$RMSE=\sqrt{\sum_{(u,i,r_{ui})\in\mathcal{R}^{te}}(r_{ui}-\hat{r}_{ui})^2/|\mathcal{R}^{te}|}$$
#### 注意
$\begin{aligned}&\text{Post-processing for G}=\{1,2,3,4,5\}\\&\text{ if }\hat{r}_{ui}>5,\text{set it as 5}\\&\text{ if }\hat{r}_{ui}<1,\text{set it as 1}\end{aligned}$
#### 结果
- Method RMSE MAE
- User-based CF 0.9554 0.7480
- Item-based CF 0.9901 0.7801
- Hybrid CF 0.9562 0.7538